In [4]:
import sqlite3

conn = sqlite3.connect("eng_subtitles_database.db")

In [5]:
import pandas as pd

query = 'SELECT * FROM zipfiles'

df = pd.read_sql_query(query, conn)

conn.close()

df.head()

,num,name,content
0,9180533,the.message.(1976).eng.1cd,b'PK\x03\x04\x14\x00\x00\x00\x08\x00\x1c\xa9\x...
1,9180583,here.comes.the.grump.s01.e09.joltin.jack.in.bo...,b'PK\x03\x04\x14\x00\x00\x00\x08\x00\x17\xb9\x...
2,9180592,yumis.cells.s02.e13.episode.2.13.(2022).eng.1cd,b'PK\x03\x04\x14\x00\x00\x00\x08\x00L\xb9\x99V...
3,9180594,yumis.cells.s02.e14.episode.2.14.(2022).eng.1cd,b'PK\x03\x04\x14\x00\x00\x00\x08\x00U\xa9\x99V...
4,9180600,broker.(2022).eng.1cd,b'PK\x03\x04\x14\x00\x00\x00\x08\x001\xa9\x99V...


In [6]:
df.shape


(82498, 3)

In [7]:
df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 82498 entries, 0 to 82497
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   num      82498 non-null  int64 
 1   name     82498 non-null  object
 2   content  82498 non-null  object
dtypes: int64(1), object(2)
memory usage: 1.9+ MB


In [8]:
from tqdm import tqdm, tqdm_notebook
tqdm.pandas()


In [9]:
import zipfile
import io
def decomp_decode(data):
    with zipfile.ZipFile(io.BytesIO(data)) as zip_file:
        # Extract the first file in the ZIP archive
        file_list = zip_file.namelist()
        first_file = file_list[0]
        decompressed_data = zip_file.read(first_file)
    return decompressed_data.decode('latin-1')

In [10]:
df['content'] = df['content'].progress_apply(lambda x : decomp_decode(x))


100%|██████████████████████████████████████████████████████████████████████████| 82498/82498 [01:07<00:00, 1218.05it/s]


In [15]:
!pip install re

Defaulting to user installation because normal site-packages is not writeable


ERROR: Could not find a version that satisfies the requirement re (from versions: none)
ERROR: No matching distribution found for re


In [20]:
import re

def clean_data(data): # data is the entire text file entry in the dataframe

    # removing timestamps
    
    data = re.sub(r"\d{2}:\d{2}:\d{2},\d{3}\s-->\s\d{2}:\d{2}:\d{2},\d{3}", " ", data)
    
    # removing index no. of dialogues
    data = re.sub(r'\n?\d+\r', "", data)

    # removing escape sequences like \n \r
    data = re.sub('\r|\n', "", data)

    # removing <i> and </i>
    data = re.sub('<i>|</i>', "", data)
    # removing links
    
    data = re.sub(r"(?:www\.)osdb\.link\/[\w\d]+|www\.OpenSubtitles\.org|osdb\.link\/ext|api\.OpenSubtitles\.org|OpenSubtitles\.com", " ", data)
    # Converting to lower case
    data = data.lower()

    # return
    return data

In [21]:
df['content'] = df['content'].progress_apply(lambda x: clean_data(x))


100%|███████████████████████████████████████████████████████████████████████████| 82498/82498 [02:21<00:00, 581.59it/s]


In [22]:
df.to_csv('new12.csv', index = True)
print("done")

done


In [24]:
df1 = pd.read_csv('new12.csv')
df1.head(10)

ParserError: Error tokenizing data. C error: out of memory